In [1]:
# import os
# import numpy as np
# # split full data into 2 mutually exclusive subfolders
# input_dir = "./../INPUT_all/data/combined_data"
# output_dir_A = "./../INPUT_all/data/combined_split_A"
# output_dir_B = "./../INPUT_all/data/combined_split_B"
# 
# # Read files into a numpy array
# file_list = os.listdir(input_dir)[1:]
# print(file_list)
# # for file in file_list
# for cat in range(10):
#     file = file_list[cat]
#     data = np.loadtxt(os.path.join(input_dir, file), dtype=str)
# 
#     split_index = len(data) // 2
#     split_A = data[:split_index]
#     split_B = data[split_index:]
#     if len(split_A) > len(split_B):
#         split_A = split_A[:len(split_B)]
#     else:
#         split_B = split_B[:len(split_A)]
# 
# 
#     np.savetxt(os.path.join(output_dir_A, f"half-images-unlabeled-{cat}.dat"), split_A, fmt="%s")
#     np.savetxt(os.path.join(output_dir_B, f"half-images-unlabeled-{cat}.dat"), split_B, fmt="%s")


In [1]:
import numpy as np
import os
import json
import sys
import matplotlib.pyplot as plt

sys.path.append("../")

In [2]:
import importlib
import src.plot as myplot
importlib.reload(myplot)

<module 'src.plot' from '/Users/paulhosek/PycharmProjects/mcm/MCM_classifier/Classifier_1/0_classifier/paper_plots/../src/plot.py'>

In [109]:
# XXX_sample => [sample size, run_index, cat_idx, icc_idx]
def load_counts_mcm(digit, nr_runs, sample_sizes, letter):
    """For all sample sizes, load the counts and mcms for a specific digit and a pool of samples specifid by the letter."""
    samples_path = "../OUTPUT/sample_sizes_split_{}".format(letter)

    fname = "Counts_"
    counts_sample = []
    for s, sample_size in enumerate(sample_sizes):
        counts_runs = []
        counts_path = os.path.join(samples_path, str(sample_size), "Counts")
        for i in range(len(os.listdir(counts_path))):
            with open(os.path.join(counts_path, fname+str(i)+ ".json")) as f:
                    counts_runs.append(json.load(f))
        
        counts_sample.append(counts_runs)

                    
    fname = "MCMs_"
    mcm_sample = []
    for s, sample_size in enumerate(sample_sizes):
        mcm_runs = []
        mcms_path = os.path.join(samples_path, str(sample_size), "MCMs")
        for i in range(len(os.listdir(mcms_path))):
            with open(os.path.join(mcms_path, fname+str(i)+ ".json")) as f:
                    # samples[sample_size] = json.load(f)
                    mcm_runs.append(json.load(f))
        mcm_sample.append(mcm_runs)


    return counts_sample, mcm_sample

# counts_sample, mcm_sample = load_counts_mcm(0,10,[10,100],"B")

In [110]:
# get sample seed=42 from dataset B
def recreate_dataset(sample_from_letter, sample_size:int, seed = 42):
    """Recreate the dataset A or B was build on."""
    all_data_path="../INPUT_all/data/combined_split_{}".format(sample_from_letter)
    file = "half-images-unlabeled-0.dat"
    sample_size_from_letter = sample_size # needs to be the same as build from sample size. We just show it the exact samples the other one is build on.
    rng = np.random.default_rng(seed=seed)
    inp = np.loadtxt(os.path.join(all_data_path,file), dtype="str")
    return rng.choice(inp, sample_size_from_letter,replace=False)

sample_B = recreate_dataset("B", 10)

In [179]:
# average probability for each icc for observing the data B of the same size was build on
# do the same for observing A
def generate_counts_ranks_singlerun_singlesample(counts_sample, mcm_sample, see_data,run_idx, sample_idx, cat_idx = 0):
    """Generate the observed counts for iccs to observe the "see_data" sample. Also returns the ranks of the iccs.
    That is it returns a list of count distributions over samples for every icc.
    """




    sum_of_count = np.sum(counts_sample[sample_idx][run_idx][cat_idx][0])
    data = np.array([[int(s) for s in state] for state in see_data])
    nr_icc = len(counts_sample[sample_idx][run_idx][cat_idx])
    counts_observe_B = np.empty((nr_icc, see_data.shape[0]))
    for icc in range(nr_icc):
        counts_observe_B[icc,:] = (myplot.calc_p_icc_single(data,counts_sample[sample_idx][run_idx][cat_idx],121,mcm_sample[sample_idx][run_idx][cat_idx],icc))

    ranks = np.genfromtxt(mcm_sample[sample_idx][run_idx][cat_idx],dtype=int,delimiter=1).sum(axis=1)

    return counts_observe_B, ranks




In [180]:
def counts_to_prob(icc_pdf, rank,sum_of_count, add_smooth=True):
    alpha = 1
    if add_smooth:
        zero_f =  lambda x: (x+alpha)/ (sum_of_count+rank*alpha) 
    else:
        zero_f = lambda x: x/sum_of_count

    return np.apply_along_axis(zero_f,0,icc_pdf)

# calculate probabilities P(MCM_A see data_B)
def probs_mean_std(counts_observe_X, ranks, sum_of_count, sample_size,n_icc):
    """Calculate mean and std probability of mcm to observe some data over samples.
    That is mean over runs (P(mcm_digitx_samplesizex observes data X). 
    e.g., mcm digit 0 using 100 samples from pool A to build it sees 100 samples from pool B"""

    probs = np.empty((n_icc,sample_size))
    for i in range(len(ranks)):
        probs[i,:] = counts_to_prob(counts_observe_X[i],ranks[i], sum_of_count)
    probs_mcm = np.product(probs,axis=0)

    return np.mean(probs_mcm), np.std(probs_mcm)



In [181]:
# calculate mean and std for seeing recreated sample for every mcm
# P()

digit = 0
nr_runs = 10
sample_sizes = [10,100]
counts_sample, mcm_sample = load_counts_mcm(digit,nr_runs,[10,100],"B")
letter = "B"

ms_all = np.empty((len(sample_sizes),nr_runs,2))
for sample_idx, sample_size in enumerate(sample_sizes):
    m_s_run = np.empty((nr_runs,2))
    n_icc = len(counts_sample[sample_idx][0][digit])
    for run_idx in range(nr_runs):
        sample_recreate = recreate_dataset(letter, sample_size)

        counts_observe_X, ranks = generate_counts_ranks_singlerun_singlesample(counts_sample, mcm_sample, sample_recreate, run_idx, sample_idx)
        sum_of_count = np.sum(counts_sample[sample_idx][run_idx][0][0])
        # mean probabilities over 100 B samples for some mcm
        m,s = probs_mean_std(counts_observe_X, ranks, sum_of_count,sample_size, n_icc) 
        ms_all[sample_idx, run_idx, :] = [m,s]


np.mean(ms_all,axis=1)

array([[5.99128967e-13, 1.05541537e-12],
       [3.18626805e-13, 6.66846961e-13]])

array([[5.99128967e-13, 1.05541537e-12],
       [3.18626805e-13, 6.66846961e-13]])

In [142]:
# probs = np.empty((n_icc,sample_size))
# for i in range(len(ranks)):
#     probs[i,:] = counts_to_prob(counts_observe_X[i],ranks[i], sum_of_count)
# probs_mcm = np.product(probs,axis=0)

# # return np.mean(probs_mcm), np.std(probs_mcm)

In [144]:
# print(probs_mcm.shape)

(100,)


In [38]:
# calculate average probability for digit 0 for 10 samples sample idx 0 for seeing a 0 of dataset B


In [ ]:
# now working with A only
# for every sample size
# for every sample, need to calculate average probability of seeing seed=42 sample in dataset B
# then we calculate the mean + std over the 50 samples in a sample size

# do the same with B

In [9]:
n_variables = 121
data_path = "../INPUT_all/data/"
data_filename_format = "train-images-unlabeled-{}.dat"
sample_size = 10
C = counts_sample[0]
all_MCM = mcm_sample[0]

# # for all images in all categories, get the PDF of that all ICC of one MCM fitted to one image
all_C_icc = myplot.calculate_P_icc(C, all_MCM, 0, n_variables, data_path, data_filename_format)


np.save(samples_path + f"/{sample_size}/all_C_icc_c{digit}_s{sample_idx}.npy", all_C_icc)

del all_C_icc

FileNotFoundError: [Errno 2] No such file or directory: '../OUTPUT/sample_sizes_split_A10/all_C_icc_0.npy'

In [14]:
all_C_icc_c0_s0 = np.load(samples_path + f"/{sample_size}/all_C_icc_c0_s0.npy")

In [16]:
all_C_icc_c0_s0.shape

(21, 5423, 10)

In [ ]:
mcms_fname = "MCMs_0.json"
counts_fname = "Counts_0.json"
n_samples = 100

with open(os.path.join(samples_path,str(n_samples),"MCMs", mcms_fname), "r") as f:
        all_MCM = [np.array(i) for i in json.load(f)] 


with open(os.path.join(samples_path,str(n_samples),"Counts", counts_fname), "r") as f:
        C = json.load(f

In [ ]:
n_variables = 121 #TODO This cell is not trash.
data_path = "../INPUT_all/data/"
data_filename_format = "train-images-unlabeled-{}.dat"

# # for all images in all categories, get the PDF of that all ICC of one MCM fitted to one image
all_C_icc_3 = myplot.calculate_P_icc(C, all_MCM, 3, n_variables, data_path, data_filename_format)
np.save("../OUTPUT/sample_sizes/5423/all_C_icc_3.npy", all_C_icc_3)


all_C_icc_3 = np.load("../OUTPUT/sample_sizes/5423/all_C_icc_3.npy")